In [6]:
import spacy
from sklearn.feature_extraction.text import CountVectorizer
nlp= spacy.load('en_core_web_sm')

In [35]:
# convert this text into a vector #using BOW
corpus= ["you have faith in god", "you are so beautiful"]

v= CountVectorizer()

t= v.fit_transform(corpus)

a= t.toarray()
a

array([[0, 0, 1, 1, 1, 1, 0, 1],
       [1, 1, 0, 0, 0, 0, 1, 1]], dtype=int64)

In [37]:
# convert this text into a vector #using n gram

corpus= ['you have faith in god']
v= CountVectorizer(ngram_range=(2, 2))
v.fit_transform(corpus)

v.vocabulary_


{'you have': 3, 'have faith': 1, 'faith in': 0, 'in god': 2}

In [39]:
#Combination of 1 and 2 gram
v= CountVectorizer(ngram_range=(1, 2))
v.fit_transform(corpus)

v.vocabulary_

{'you': 7,
 'have': 3,
 'faith': 0,
 'in': 5,
 'god': 2,
 'you have': 8,
 'have faith': 4,
 'faith in': 1,
 'in god': 6}

News classification

In [40]:
import pandas as pd

In [47]:
data= pd.read_json('resources/News_Category_Dataset_v3.json', lines= True)
data.head()

,link,headline,category,short_description,authors,date
0,https://www.huffpost.com/entry/covid-boosters-...,Over 4 Million Americans Roll Up Sleeves For O...,U.S. NEWS,Health experts said it is too early to predict...,"Carla K. Johnson, AP",2022-09-23
1,https://www.huffpost.com/entry/american-airlin...,"American Airlines Flyer Charged, Banned For Li...",U.S. NEWS,He was subdued by passengers and crew when he ...,Mary Papenfuss,2022-09-23
2,https://www.huffpost.com/entry/funniest-tweets...,23 Of The Funniest Tweets About Cats And Dogs ...,COMEDY,"""Until you have a dog you don't understand wha...",Elyse Wanshel,2022-09-23
3,https://www.huffpost.com/entry/funniest-parent...,The Funniest Tweets From Parents This Week (Se...,PARENTING,"""Accidentally put grown-up toothpaste on my to...",Caroline Bologna,2022-09-23
4,https://www.huffpost.com/entry/amy-cooper-lose...,Woman Who Called Cops On Black Bird-Watcher Lo...,U.S. NEWS,Amy Cooper accused investment firm Franklin Te...,Nina Golgowski,2022-09-22


In [53]:
data= data.drop(columns=['link', 'headline', 'authors', 'date'])

In [55]:
data.category.value_counts()

POLITICS          35602
WELLNESS          17945
ENTERTAINMENT     17362
TRAVEL             9900
STYLE & BEAUTY     9814
PARENTING          8791
HEALTHY LIVING     6694
QUEER VOICES       6347
FOOD & DRINK       6340
BUSINESS           5992
COMEDY             5400
SPORTS             5077
BLACK VOICES       4583
HOME & LIVING      4320
PARENTS            3955
THE WORLDPOST      3664
WEDDINGS           3653
WOMEN              3572
CRIME              3562
IMPACT             3484
DIVORCE            3426
WORLD NEWS         3299
MEDIA              2944
WEIRD NEWS         2777
GREEN              2622
WORLDPOST          2579
RELIGION           2577
STYLE              2254
SCIENCE            2206
TECH               2104
TASTE              2096
MONEY              1756
ARTS               1509
ENVIRONMENT        1444
FIFTY              1401
GOOD NEWS          1398
U.S. NEWS          1377
ARTS & CULTURE     1339
COLLEGE            1144
LATINO VOICES      1130
CULTURE & ARTS     1074
EDUCATION       

In [59]:
#lets focus only on the top 5 categories
options= ['POLITICS', 'WELLNESS', 'ENTERTAINMENT','TRAVEL', 'STYLE & BEAUTY']
data = data.loc[data.category.isin(options)]
data

,category,short_description
20,ENTERTAINMENT,"For the past 18 months, Hollywood has effectiv..."
21,POLITICS,President issues vow as tensions with China rise.
24,POLITICS,An annual celebration took on a different feel...
28,ENTERTAINMENT,"The ""Avatar"" director said aspects of his 2009..."
30,POLITICS,"U.S. President Joe Biden, in London for the fu..."
...,...,...
209478,TRAVEL,"As we stepped out of the serene plantation, we..."
209479,WELLNESS,"My great-aunt Ida loves to say, ""This is only ..."
209480,STYLE & BEAUTY,"""A Look Back"" is a daily column that highlight..."
209512,ENTERTAINMENT,Representation of the collective diaspora has ...


In [83]:
#a lot of imbalance in the  data
data.category.value_counts()

POLITICS          35602
WELLNESS          17945
ENTERTAINMENT     17362
TRAVEL             9900
STYLE & BEAUTY     9814
Name: category, dtype: int64

In [61]:
# we can do under sampling 
#1) select the minimum wala section (style and beauty (9814)
#2) random select 9814 samples from the other categories

In [84]:
df_poli= data[data.category== 'POLITICS'].sample(random_state= 200, n= 9814)
df_well= data[data.category== 'WELLNESS'].sample(random_state= 200, n= 9814)
df_ent= data[data.category== 'ENTERTAINMENT'].sample(random_state= 200, n= 9814)
df_trav= data[data.category== 'TRAVEL'].sample(random_state= 200, n= 9814)
df_styl= data[data.category== 'STYLE & BEAUTY'].sample(random_state= 200, n= 9814)

In [94]:
df= pd.concat([df_poli, df_well, df_ent, df_trav, df_styl], axis= 0)
df.category.value_counts()

POLITICS          9814
WELLNESS          9814
ENTERTAINMENT     9814
TRAVEL            9814
STYLE & BEAUTY    9814
Name: category, dtype: int64

In [95]:
df.head(5)

,category,short_description
9453,POLITICS,So much for an all-out assault on Big Pharma.
58721,POLITICS,Trump’s newly appointed campaign manager was p...
84240,POLITICS,Investigators are reportedly scrutinizing whet...
5560,POLITICS,The president dismissed the country's hints at...
37390,POLITICS,We have worked too hard and sacrificed too muc...


In [103]:
#you need to replace these text labels with numbers encoding

from sklearn.preprocessing import LabelEncoder
l= LabelEncoder()
df.category= l.fit_transform(df.category)
df.head(5)

,category,short_description
9453,1,So much for an all-out assault on Big Pharma.
58721,1,Trump’s newly appointed campaign manager was p...
84240,1,Investigators are reportedly scrutinizing whet...
5560,1,The president dismissed the country's hints at...
37390,1,We have worked too hard and sacrificed too muc...


In [104]:
df.category.value_counts()

1    9814
4    9814
0    9814
3    9814
2    9814
Name: category, dtype: int64

In [108]:
df

,category,short_description
9453,1,So much for an all-out assault on Big Pharma.
58721,1,Trump’s newly appointed campaign manager was p...
84240,1,Investigators are reportedly scrutinizing whet...
5560,1,The president dismissed the country's hints at...
37390,1,We have worked too hard and sacrificed too muc...
...,...,...
200102,2,"A former employee of Chris Burch, Tory's ex an..."
203065,2,But Kelly Osbourne truly stepped up to the pla...
206135,2,"At first glance, raffia may appear to be a fab..."
139960,2,Kelly Rowland While there were certainly some ...


In [113]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test= train_test_split(df.short_description, df.category, test_size= 0.2, stratify= df.category, random_state= 200)

In [114]:
#stratify
y_test.value_counts()

1    1963
2    1963
4    1963
0    1963
3    1962
Name: category, dtype: int64

In [129]:
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics import classification_report

## Without any preprocessing

In [137]:
#this is the BOW MODEL (1 GRAM)
from sklearn.naive_bayes import MultinomialNB

clf= Pipeline([
    ('vectorizer', CountVectorizer()),
    ('nb', MultinomialNB())
])

clf.fit(x_train, y_train)

y_pred= clf.predict(x_test)

print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.75      0.44      0.56      1963
           1       0.82      0.69      0.75      1963
           2       0.81      0.77      0.79      1963
           3       0.64      0.81      0.71      1962
           4       0.66      0.88      0.76      1963

    accuracy                           0.72      9814
   macro avg       0.74      0.72      0.71      9814
weighted avg       0.74      0.72      0.71      9814



In [140]:
# THIS IS THE COMBINATION OF BOW(1 RGAM) AND 2 GRAM MODEL
from sklearn.naive_bayes import MultinomialNB

clf= Pipeline([
    ('vectorizer', CountVectorizer(ngram_range=(1, 2))),
    ('nb', MultinomialNB())
])

clf.fit(x_train, y_train)

y_pred= clf.predict(x_test)

print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.87      0.32      0.46      1963
           1       0.86      0.62      0.72      1963
           2       0.79      0.75      0.77      1963
           3       0.60      0.82      0.69      1962
           4       0.58      0.92      0.71      1963

    accuracy                           0.69      9814
   macro avg       0.74      0.69      0.67      9814
weighted avg       0.74      0.69      0.67      9814



## AFTER PREPROCESSING

In [144]:
# remove stop words, remove punct, convert in base word

def preprocess(text):
    doc= nlp(text)

    filter= []

    for token in doc:
        if not token.is_stop and not token.is_punct:
            filter.append(token.lemma_) 

    return ' '.join(filter)        

In [146]:
df['short_description_new'] = df['short_description'].apply(preprocess) 

In [148]:
df.head(5)

,category,short_description,short_description_new
9453,1,So much for an all-out assault on Big Pharma.,assault Big Pharma
58721,1,Trump’s newly appointed campaign manager was p...,Trump newly appoint campaign manager press pla...
84240,1,Investigators are reportedly scrutinizing whet...,investigator reportedly scrutinize company mis...
5560,1,The president dismissed the country's hints at...,president dismiss country hint missile test sa...
37390,1,We have worked too hard and sacrificed too muc...,work hard sacrifice treat like help


In [149]:
x_train, x_test, y_train, y_test= train_test_split(df.short_description_new, df.category, test_size= 0.2, stratify= df.category, random_state= 200)

In [150]:
x_train

184397    ton space site party planner hotel host devili...
120972    football great sport people crave structure   ...
149361    Kindness Incident 1 day take hot yoga class No...
168590    good thing New York City inspire people street...
92878        Cara Delevingne exist feed narrative news feed
                                ...                        
144092    source Benson Henry Institute 2 engage relaxat...
27338                                    go Star Wars movie
156730    quit caffeine easy feel consumption bad habit ...
157680    thirteen year apply Fulbright program month be...
1310      follow simple expert back hack inject a.m. lit...
Name: short_description_new, Length: 39256, dtype: object

In [151]:
#this is the BOW MODEL (1 GRAM)
from sklearn.naive_bayes import MultinomialNB

clf= Pipeline([
    ('vectorizer', CountVectorizer()),
    ('nb', MultinomialNB())
])

clf.fit(x_train, y_train)

y_pred= clf.predict(x_test)

print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.74      0.46      0.56      1963
           1       0.81      0.71      0.76      1963
           2       0.80      0.79      0.79      1963
           3       0.63      0.82      0.71      1962
           4       0.71      0.87      0.78      1963

    accuracy                           0.73      9814
   macro avg       0.74      0.73      0.72      9814
weighted avg       0.74      0.73      0.72      9814



In [155]:
# THIS IS THE COMBINATION OF BOW(1 RGAM) AND 2 GRAM MODEL
from sklearn.naive_bayes import MultinomialNB

clf= Pipeline([
    ('vectorizer', CountVectorizer(ngram_range=(1, 2))),
    ('nb', MultinomialNB())
])

clf.fit(x_train, y_train)

y_pred= clf.predict(x_test)

print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.83      0.35      0.49      1963
           1       0.84      0.67      0.75      1963
           2       0.79      0.80      0.80      1963
           3       0.61      0.84      0.71      1962
           4       0.66      0.90      0.76      1963

    accuracy                           0.72      9814
   macro avg       0.75      0.72      0.70      9814
weighted avg       0.75      0.72      0.70      9814

